# Final Project

## Notes

In [ ]:
'''
Using the following machine learning models for making predictions for the following portfolio optimization models:
- Mean-variance optimization (chapter 7 of Bodie, Investment book)
- Index model (chapter 8 of Bodie, Investment book)
- Capital asset pricing model (chapter 9 of Bodie, Investment book)
- Arbitrage pricing theory and multifactor model (chapter 10 of Bodie,
Investment book)
- Equity valuation model (chapter 18 of Bodie, Investment book)
- Black Litterman model (chapter 24 of Bodie, Investment book)
- Algorithmic trading (this could be restricted to the last month)
'''

## Load Data

In [5]:
import yfinance as yf
from pypfopt import plotting
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from pypfopt import expected_returns, risk_models, EfficientFrontier
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

tickers = ["AAPL", "AMZN", "BRK-B", "GOOGL", "META", "MSFT", "NVDA", "TSLA", "UNH", "XOM"]
portfolio_data = yf.download(tickers, period="5y")
portfolio_data = portfolio_data["Adj Close"]
portfolio_data.tail()

[*********************100%***********************]  10 of 10 completed


,AAPL,AMZN,BRK-B,GOOGL,META,MSFT,NVDA,TSLA,UNH,XOM
Date,,,,,,,,,,
2023-06-26,185.270004,127.330002,334.119995,118.339996,278.470001,328.600006,406.320007,241.050003,479.059998,104.290001
2023-06-27,188.059998,129.179993,335.339996,118.330002,287.049988,334.570007,418.760010,250.210007,482.559998,104.550003
2023-06-28,189.250000,129.039993,334.149994,120.180000,285.290009,335.850006,411.170013,256.239990,474.450012,105.400002
2023-06-29,189.589996,127.900002,336.910004,119.099998,281.529999,335.049988,408.220001,257.500000,476.440002,106.699997
2023-06-30,193.970001,130.360001,341.000000,119.699997,286.980011,340.540009,423.019989,261.769989,480.640015,107.250000


In [6]:
market_index_ticker = "^GSPC"  # S&P 500 index
market_index_data = yf.download(market_index_ticker, period="5y")
market_index_data = market_index_data["Adj Close"]
market_index_data.tail()

[*********************100%***********************]  1 of 1 completed


Date
2023-06-26    4328.819824
2023-06-27    4378.410156
2023-06-28    4376.859863
2023-06-29    4396.439941
2023-06-30    4450.379883
Name: Adj Close, dtype: float64

## Mean Variance Optimization

### Random Forest 

In [ ]:
future_prices = {}
for ticker in tickers:
    data = portfolio_data[ticker].tolist()
    # Dependent variable - 10 consecutive days of stock prices
    dev_x = [data[i:i+10] for i in range(len(data)-20)]
    
    # Independent variable - stock price 10th day into the future
    dev_y = [data[i+10] for i in range(10,len(data)-10)]
    
    test = [data[i:i+10] for i in range(len(data)-20,len(data)-10)]
    
    reg = RandomForestRegressor()
    reg.fit(dev_x,dev_y)
    
    # Predict stock price for 10 future days
    pred = reg.predict(test)
    future_prices[ticker] = pred

future_prices = pd.DataFrame(future_prices)
# Construct covariance matrix of future stock prices
S = risk_models.CovarianceShrinkage(future_prices).ledoit_wolf()

# Use capm to find expected returns on future prices
mu = expected_returns.capm_return(future_prices)
print(mu)

# Do mean variance optimization using efficient frontier
ef = EfficientFrontier(mu, S)
ef.min_volatility()
weights = ef.clean_weights()
print(weights)
# weights = ef.max_sharpe(risk_free_rate=0.02)
# cleaned_weights = ef.clean_weights()
# print(cleaned_weights)

ret, volatility, sharpe_ratio = ef.portfolio_performance()
print("Expected annual return:", ret)
print("Annual volatility:", volatility)
print("Sharpe ratio:", sharpe_ratio)


### SVM

In [ ]:
future_prices = {}
for ticker in tickers:
    data = portfolio_data[ticker].tolist()
    dev_x = [data[i:i+10] for i in range(len(data)-20)]
    dev_y = [data[i+10] for i in range(10, len(data)-10)]
    test = [data[i:i+10] for i in range(len(data)-20, len(data)-10)]
    
    reg = SVR()
    reg.fit(dev_x, dev_y)
    
    pred = reg.predict(test)
    future_prices[ticker] = pred

future_prices = pd.DataFrame(future_prices)

S = risk_models.CovarianceShrinkage(future_prices).ledoit_wolf()
mu = expected_returns.capm_return(future_prices)
print(mu)

ef = EfficientFrontier(mu, S)
ef.min_volatility()
weights = ef.clean_weights()
print(weights)


ret, volatility, sharpe_ratio = ef.portfolio_performance()
print("Expected annual return:", ret)
print("Annual volatility:", volatility)
print("Sharpe ratio:", sharpe_ratio)


### Neural Network

In [ ]:
future_prices = {}
for ticker in tickers:
    data = portfolio_data[ticker].tolist()
    dev_x = [data[i:i+10] for i in range(len(data)-20)]
    dev_y = [data[i+10] for i in range(10, len(data)-10)]
    test = [data[i:i+10] for i in range(len(data)-20, len(data)-10)]
    
    reg = MLPRegressor(hidden_layer_sizes=(100,100), random_state=42)  # You can adjust the hidden_layer_sizes and other parameters as needed
    reg.fit(dev_x, dev_y)
    
    pred = reg.predict(test)
    future_prices[ticker] = pred

future_prices = pd.DataFrame(future_prices)

S = risk_models.CovarianceShrinkage(future_prices).ledoit_wolf()
mu = expected_returns.capm_return(future_prices)
print(mu)

ef = EfficientFrontier(mu, S)
ef.min_volatility()
weights = ef.clean_weights()
print(weights)

ret, volatility, sharpe_ratio = ef.portfolio_performance()
print("Expected annual return:", ret)
print("Annual volatility:", volatility)
print("Sharpe ratio:", sharpe_ratio)



## Index model

In [ ]:
class SingleIndexModel:
    def __init__(self, tickers, market_index_data, portfolio_data, reg_model):
        self.tickers = tickers
        self.market_index_data = market_index_data
        self.portfolio_data = portfolio_data
        self.reg_model = reg_model
        self.returns = {}
        self.beta = {}
        self.alpha = {}
        self.residual_variance = {}

    def calculate_returns(self):
        # Returns of stocks and market
        self.returns = {"MARKET": ((self.market_index_data / self.market_index_data.shift(1)) - 1).dropna().tolist()}
        for ticker in self.tickers:
            self.returns[ticker] = ((self.portfolio_data[ticker] / self.portfolio_data[ticker].shift(1))-1).dropna().tolist()

    def fit_regression(self):
        # Forecast market
        market_x = [self.returns["MARKET"][i:i+10] for i in range(len(self.returns["MARKET"])-20)]
        market_y = [self.returns["MARKET"][i+10] for i in range(10, len(self.returns["MARKET"])-10)]
        market_test = [self.returns["MARKET"][i:i+10] 
                       for i in range(len(self.returns["MARKET"])-20, len(self.returns["MARKET"])-10)]

        reg = self.reg_model
        reg.fit(market_x, market_y)
        market_future = reg.predict(market_test)

        # Forecast each stock
        for ticker in self.tickers:
            stock_x = [self.returns[ticker][i:i+10] for i in range(len(self.returns[ticker])-20)]
            stock_y = [self.returns[ticker][i+10] for i in range(10, len(self.returns[ticker])-10)]
            stock_test = [self.returns[ticker][i:i+10]
                          for i in range(len(self.returns[ticker])-20, len(self.returns[ticker])-10)]

            reg = self.reg_model
            reg.fit(stock_x, stock_y)
            stock_future = reg.predict(stock_test)

            single_index_reg = LinearRegression()
            single_index_reg.fit(np.array(market_future).reshape(-1, 1), y=stock_future)

            self.beta[ticker] = single_index_reg.coef_[0]
            self.alpha[ticker] = single_index_reg.intercept_

            y_pred = single_index_reg.predict(np.array(self.returns["MARKET"]).reshape(-1, 1))
            residuals = self.returns[ticker] - y_pred
            self.residual_variance[ticker] = np.var(residuals)

    def optimize_portfolio(self):
        # Compute the initial position of each security
        weights = {ticker: self.alpha[ticker] / self.residual_variance[ticker] for ticker in self.tickers}
        total_weight = sum(weights.values())
        weights = {ticker: weight / total_weight for ticker, weight in weights.items()}

        # Compute the alpha of the active portfolio
        alpha_portfolio = sum(weights[ticker] * self.alpha[ticker] for ticker in self.tickers)

        # Compute the residual variance of the active portfolio
        residual_variance_portfolio = sum((weights[ticker] ** 2) * self.residual_variance[ticker] for ticker in self.tickers)

        # Compute the initial position in the active portfolio
        residual_variance_market = 0.0114  # Variance of S&P 500
        risk_premium_market = 0.056
        initial_position_portfolio = (alpha_portfolio * residual_variance_market) / (residual_variance_portfolio * risk_premium_market)

        # Compute the beta of the active portfolio
        beta_portfolio = sum(weights[ticker] * self.beta[ticker] for ticker in self.tickers)

        # Adjust the initial position in the active portfolio
        adjusted_position_portfolio = initial_position_portfolio / (1 + (1 - beta_portfolio) * initial_position_portfolio)

        # Optimal risky portfolio now has weights
        final_weight_market = 1 - adjusted_position_portfolio
        weights = {ticker: weight * adjusted_position_portfolio for ticker, weight in weights.items()}

        # Calculate the risk premium of the portfolio
        risk_premium_portfolio = (final_weight_market + adjusted_position_portfolio * beta_portfolio) * risk_premium_market + adjusted_position_portfolio * alpha_portfolio

        # Compute the variance of the portfolio
        portfolio_variance = (final_weight_market + adjusted_position_portfolio * beta_portfolio) ** 2 * residual_variance_market + adjusted_position_portfolio ** 2 * residual_variance_portfolio

        # Calculate the Sharpe ratio
        sharpe_ratio = risk_premium_portfolio / (portfolio_variance ** 0.5)

        return weights, final_weight_market, sharpe_ratio


### Random Forest

In [ ]:
reg_model = RandomForestRegressor()
portfolio = SingleIndexModel(tickers, market_index_data, portfolio_data, reg_model)
portfolio.calculate_returns()
portfolio.fit_regression()
weights, final_weight_market, sharpe_ratio = portfolio.optimize_portfolio()
print("------------------------------")
print("Final Weights:")
print("Weight Market S&P:", final_weight_market)
print(f"Weights: {weights}")
print("------------------------------")
print("Sharpe ratio:", sharpe_ratio)

### SVM

In [ ]:
reg_model = SVR()
portfolio = SingleIndexModel(tickers, market_index_data, portfolio_data, reg_model)
portfolio.calculate_returns()
portfolio.fit_regression()
weights, final_weight_market, sharpe_ratio = portfolio.optimize_portfolio()
print("------------------------------")
print("Final Weights:")
print("Weight Market S&P:", final_weight_market)
print(f"Weights: {weights}")
print("------------------------------")
print("Sharpe ratio:", sharpe_ratio)

### Neural Network

In [ ]:
reg_model = MLPRegressor(hidden_layer_sizes=(100,100))
portfolio = SingleIndexModel(tickers, market_index_data, portfolio_data, reg_model)
portfolio.calculate_returns()
portfolio.fit_regression()
weights, final_weight_market, sharpe_ratio = portfolio.optimize_portfolio()
print("------------------------------")
print("Final Weights:")
print("Weight Market S&P:", final_weight_market)
print(f"Weights: {weights}")
print("------------------------------")
print("Sharpe ratio:", sharpe_ratio)

### Capital asset pricing model (CAPM)

In [ ]:
returns = {"MARKET": ((market_index_data / market_index_data.shift(1)) - 1).dropna().tolist()}
for ticker in tickers:
    returns[ticker] = ((portfolio_data[ticker] / portfolio_data[ticker].shift(1))-1).dropna().tolist()

print(returns.keys())

In [ ]:
market_x = [returns["MARKET"][i:i+10] for i in range(len(returns["MARKET"])-20)]
market_y = [returns["MARKET"][i+10] for i in range(10, len(returns["MARKET"])-10)]
market_test = [returns["MARKET"][i:i+10]
               for i in range(len(returns["MARKET"])-20, len(returns["MARKET"])-10)]

reg = MLPRegressor(hidden_layer_sizes=(100,100))
reg.fit(market_x, market_y)
market_future = reg.predict(market_test)
print(market_future)

In [ ]:
future_returns = {}
for ticker in tickers:
    single_index_reg = Ridge()
    single_index_reg.fit(np.array(returns["MARKET"]).reshape(-1, 1), y=returns[ticker])
    future_returns[ticker] = single_index_reg.predict(np.array(market_future).reshape(-1, 1))

future_returns = pd.DataFrame(future_returns)
print(future_returns)

In [ ]:
S = risk_models.sample_cov(future_returns)
S = risk_models.CovarianceShrinkage(future_returns).ledoit_wolf()
plotting.plot_covariance(S, plot_correlation=True)
# You don't have to provide expected returns in this case


ef = EfficientFrontier(None, S)
ef.min_volatility()
weights = ef.clean_weights()
print(weights)
# weights = ef.max_sharpe(risk_free_rate=-0.2)
# cleaned_weights = ef.clean_weights()
# print(cleaned_weights)
# print(mu)

ret, volatility, sharpe_ratio = ef.portfolio_performance()
print("Expected annual return:", ret)
print("Annual volatility:", volatility)
print("Sharpe ratio:", sharpe_ratio)

In [ ]:
from pypfopt import expected_returns
from pypfopt import plotting
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge
from pypfopt import expected_returns, risk_models, EfficientFrontier

# T-bill rate
risk_free_rate = 0.01

# apple_returns = (
#     (portfolio_data["AAPL"] / portfolio_data["AAPL"].shift(1))-1-risk_free_rate).dropna().tolist()
# google_returns = (
#     (portfolio_data["GOOGL"] / portfolio_data["GOOGL"].shift(1))-1-risk_free_rate).dropna().tolist()
# market_returns = (
#     (market_index_data / market_index_data.shift(1))-1-risk_free_rate).dropna().tolist()

# returns = {"MARKET": ((market_index_data / market_index_data.shift(1)) - 1).dropna().tolist()}
# for ticker in self.tickers:
#     returns[ticker] = ((portfolio_data[ticker] / portfolio_data[ticker].shift(1))-1).dropna().tolist()





# apple_prices = apple_data['Close'].tolist()
# google_prices = google_data['Close'].tolist()
market_x = [returns["MARKET"][i:i+10] for i in range(len(returns["MARKET"])-20)]
market_y = [returns["MARKET"][i+10] for i in range(10, len(returns["MARKET"])-10)]
market_test = [returns["MARKET"][i:i+10]
               for i in range(len(returns["MARKET"])-20, len(returns["MARKET"])-10)]

# reg = MLPRegressor(hidden_layer_sizes=(100,100))
# reg = SVR()
reg = RandomForestRegressor()

reg.fit(market_x, market_y)
market_future = reg.predict(market_test)

# print(market_future)

single_index_reg = Ridge()
single_index_reg.fit(np.array(market_returns).reshape(-1, 1), y=apple_returns)
apple_future = single_index_reg.predict(np.array(market_future).reshape(-1, 1))

single_index_reg = Ridge()
single_index_reg.fit(np.array(market_returns).reshape(-1, 1), google_returns)
google_future = single_index_reg.predict(
    np.array(market_future).reshape(-1, 1))

future_returns = {'apple': apple_future, 'google': google_future}
future_returns = pd.DataFrame(future_returns)

print(future_returns)

S = risk_models.sample_cov(future_returns)
# S = risk_models.CovarianceShrinkage(future_returns).ledoit_wolf()
# plotting.plot_covariance(S, plot_correlation=True)
# You don't have to provide expected returns in this case


ef = EfficientFrontier(None, S)
ef.min_volatility()
weights = ef.clean_weights()
print(weights)
# weights = ef.max_sharpe(risk_free_rate=-0.2)
# cleaned_weights = ef.clean_weights()
# print(cleaned_weights)
# print(mu)



### Arbitrage pricing theory and multifactor model

In [7]:
# from pypfopt import expected_returns
# from pypfopt import plotting
# import numpy as np
# import pandas as pd
# from sklearn.neural_network import MLPRegressor


# Example tickers for S&P 500, and 20+ Year Treasury Bond ETFs
factor_tickers = ['SPY', 'TLT']

# Fetch historical data for the macroeconomic factors S&P and Treasury Bond
factor_data = yf.download(factor_tickers, period="5y")

factor_data = factor_data['Adj Close']

# Calculate the returns for the S&P and Treasury bond factors
factor_returns = factor_data.pct_change().dropna()
spy_returns = factor_returns['SPY']
tlt_returns = factor_returns['TLT']

# FACTOR 1
# Train ML model to predict future price of S&P market factors
spy_x = [spy_returns[i:i+10] for i in range(len(spy_returns)-20)]
spy_y = [spy_returns[i+10] for i in range(10, len(spy_returns)-10)]

spy_test = [spy_returns[i:i+10]
               for i in range(len(spy_returns)-20, len(spy_returns)-10)]
reg = MLPRegressor(hidden_layer_sizes=(100, 100))
reg.fit(spy_x, spy_y)
spy_future = reg.predict(spy_test)

# FACTOR 2
# Train ML model to predict future price of Treasury Bond ETF factors
tlt_x = [tlt_returns[i:i+10] for i in range(len(tlt_returns)-20)]
tlt_y = [tlt_returns[i+10] for i in range(10, len(tlt_returns)-10)]

tlt_test = [tlt_returns[i:i+10]
               for i in range(len(tlt_returns)-20, len(tlt_returns)-10)]

reg = MLPRegressor(hidden_layer_sizes=(100, 100))
reg.fit(tlt_x, tlt_y)
tlt_future = reg.predict(tlt_test)

# FACTOR 3
# GDP Price of USA for past 5 years

# US GDP per capita Prices of last 5 years
GDP_prices = {
        2018: 59607,
        2019: 60698,
        2020: 58453,
        2021: 61855,
        2022: 62551,
        2023: 63451 # Forecast data also available online
    }

# Normalize the GDP values as it will make the other factors irrelevant as it is very large
values = list(GDP_prices.values())
values_array = [[value] for value in values]
scaler = StandardScaler()
scaled_values = scaler.fit_transform(values_array)
scaled_values = scaled_values.flatten()
scaled_GDP_prices = {year: scaled_value for year, scaled_value in zip(GDP_prices.keys(), scaled_values)}

print(scaled_GDP_prices)

for index, row in factor_returns.iterrows():
    # Extract the year from the date
    year = index.year

    # Fill the 'GDP' column with the corresponding GDP price based on the year
    factor_returns.at[index, 'GDP'] = scaled_GDP_prices.get(year)

future_factors = {'SPY': spy_future, 'TLT': tlt_future, 'GDP': [scaled_GDP_prices.get(2023)]*10}
future_factors = pd.DataFrame(future_factors)
#print(future_factors)

# 𝑟=𝐸(𝑟)+𝛽1𝐹1+𝛽2𝐹2+𝛽3𝐹3+𝑒 : Fit linear model to find the betas for the 3 factors chosen
# Fit Multi Factor Linear Regression model using Macroeconomic factors as X and stock returns as y
# apple_returns = (
#     (portfolio_data["AAPL"] / portfolio_data["AAPL"].shift(1))-1).dropna().tolist()
# google_returns = (
#     (portfolio_data["GOOGL"] / portfolio_data["GOOGL"].shift(1))-1).dropna().tolist()

# print(f"APPLE RETURNS: {apple_returns[0:10]}")
# print(f"GOOGLE RETURNS: {google_returns[0:10]}")

returns = {}
future_returns = {}
for ticker in tickers:
    stock_returns = ((portfolio_data[ticker] / portfolio_data[ticker].shift(1))-1).dropna().tolist()
    returns[ticker] = stock_returns
    
    single_index_reg = Ridge()
    single_index_reg.fit(factor_returns, stock_returns)
    stock_future = single_index_reg.predict(future_factors)
    future_returns[ticker] = stock_future
    print("Coefficients: ", single_index_reg.coef_)


# single_index_reg = Ridge()
# single_index_reg.fit(factor_returns, apple_returns)
# apple_future = single_index_reg.predict(future_factors)
# print("Coefficients for Apple stock: ", single_index_reg.coef_)

# single_index_reg = Ridge()
# single_index_reg.fit(factor_returns, google_returns)
# google_future = single_index_reg.predict(
#     future_factors)
# print("Coefficients for Google stock: ", single_index_reg.coef_)

# future_returns = {'apple': apple_future, 'google': google_future}




# print(list(zip(apple,google)))
future_returns = pd.DataFrame(future_returns)
print(future_returns)

S = risk_models.sample_cov(future_returns)
print(S)
# S = risk_models.CovarianceShrinkage(future_returns).ledoit_wolf()
# plotting.plot_covariance(S, plot_correlation=True)
# You don't have to provide expected returns in this case

print(future_returns.mean())
ef = EfficientFrontier(future_returns.mean(), S)
# ef.min_volatility()
# weights = ef.clean_weights()
# print(weights)
weights = ef.max_sharpe(risk_free_rate=0.0)
cleaned_weights = ef.clean_weights()
print(cleaned_weights)


[*********************100%***********************]  2 of 2 completed
{2018: -0.8731875620129659, 2019: -0.23617811356352036, 2020: -1.5469812407578423, 2021: 0.43936719519542417, 2022: 0.8457453584592317, 2023: 1.3712343626796726}
Coefficients:  [ 0.23246483 -0.03089709 -0.00044608]
Coefficients:  [ 0.21129307 -0.0063862  -0.00060686]
Coefficients:  [ 1.58778199e-01 -4.07203433e-02  8.75369524e-05]
Coefficients:  [ 2.14849351e-01 -2.47570555e-02 -1.17051810e-04]
Coefficients:  [ 0.24144847 -0.01881288  0.0003029 ]
Coefficients:  [ 0.22608141 -0.02379256 -0.00022933]
Coefficients:  [ 0.32829186 -0.02308871  0.00043708]
Coefficients:  [ 0.27754668 -0.02512154 -0.0027244 ]
Coefficients:  [ 0.16541739 -0.03677604 -0.00030013]
Coefficients:  [ 0.16781951 -0.07024288  0.00121478]
       AAPL      AMZN     BRK-B     GOOGL      META      MSFT      NVDA  \
0  0.000979 -0.000097  0.000887  0.000828  0.001351  0.001096  0.003034   
1  0.000625 -0.000397  0.000626  0.000505  0.000996  0.000758  0.

/opt/homebrew/lib/python3.10/site-packages/pypfopt/risk_models.py:70: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.warn(


In [8]:
ret, volatility, sharpe_ratio = ef.portfolio_performance()
print("Expected annual return:", ret)
print("Annual volatility:", volatility)
print("Sharpe ratio:", sharpe_ratio)


Expected annual return: 0.0018308891149965197
Annual volatility: 0.22483001293925864
Sharpe ratio: 0.008143437306527057


/opt/homebrew/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(


### Equity valuation model

### Black Litterman model 

### Algorithmic trading

In [ ]:
import numpy as np
import pandas as pd
import talib
from sklearn.ensemble import RandomForestClassifier


apple_prices = apple_data['Close']

# Calculate Bollinger Bands for Apple
apple_bb_upper, apple_bb_middle, apple_bb_lower = talib.BBANDS(
    apple_prices, timeperiod=20)


# Calculate the Bollinger Bands percentages for Apple
apple_bb_percentage = (apple_prices - apple_bb_lower) / \
    (apple_bb_upper - apple_bb_lower)

# # Combine the Bollinger Bands percentages into a single DataFrame
# bb_percentages = pd.concat(
#     [apple_bb_percentage, google_bb_percentage], axis=1).dropna()

# Define the features and target variable for the machine learning model
# Independent variables (Bollinger Bands percentages for the past period)
X = apple_bb_percentage.values[:-1]
# Target variable (-1 for sell, 1 for buy)
y = np.where(apple_prices.values[1:] > apple_bb_upper[:-1], -1, 1)

# Train a machine learning model using random forest classifier
model = RandomForestClassifier(n_estimators=100)
model.fit(X, y)

# Use the trained model to predict the trading signals for the testing set
# Ideally this will be your real time stock prices
y_pred = model.predict(X_test)

# Perform algorithmic trading based on the predicted signals (example logic)
capital = 100000  # Initial capital in USD
position = 0  # Current position (0 for neutral, 1 for long, -1 for short)

for i in range(len(y_pred)):
    if y_pred[i] == 1 and position != 1:  # Buy signal
        position = 1
        # Place a buy order based on your trading platform's API or logic

    elif y_pred[i] == -1 and position != -1:  # Sell signal
        position = -1
        # Place a sell order based on your trading platform's API or logic

    elif y_pred[i] == 0 and position != 0:  # Exit position
        position = 0
        # Close the existing position based on your trading platform's API or logic

# Calculate the final capital after the trading period
final_capital = capital  # Assume no transaction costs or slippage
# Calculate the final capital based on your trading platform's API or logic

print("Final Capital:", final_capital)
